In [1]:
from sklearn.externals import joblib
import os
import json
import numpy as np
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from collections import OrderedDict
from collections import defaultdict
from collections import Counter
from gensim.models import Word2Vec

In [2]:
predictiondict_1 = joblib.load(open("labeled_data/predictions_tablets_EN.p", "rb"))

In [3]:
predictiondict_2 = joblib.load(open("labeled_data/predictions_automobiles_EN.p", "rb"))

In [7]:
tablets_automobiles = ["tablets_EN_extra_data","automobiles_EN_extra_data"]

In [8]:
raw_data_list = []

for dirfile in tablets_automobiles:
        print("collecting comments from: ",dirfile)    
        for filename in os.listdir(dirfile):
            file = dirfile+"/"+filename
            if file[-4:] == "json":
                with open(file) as json_data:
                    d = json.load(json_data)
                    for comment in d["comments"]:
                        lower = comment["text"].lower().strip()
                        raw_data_list.append([comment["id"],d["video_id"],lower])

collecting comments from:  D:/Thesis/Make_normal_Embeddings/tablets_EN_extra_data
collecting comments from:  D:/Thesis/Make_normal_Embeddings/automobiles_EN_extra_data


In [9]:
print(len(predictiondict_2))

1028136


In [10]:
predictiondict = {}

In [11]:
predictiondict.update(predictiondict_2)

In [12]:
print(len(predictiondict))

1028136


In [13]:
predictiondict.update(predictiondict_1)

In [14]:
print(len(predictiondict))

1610828


In [15]:
# create commentlists of positive, negative and neutral comments

labellist_pos_test = []
commentlist_pos_test =[]
labellist_neg_test = []
commentlist_neg_test =[]
labellist_neu_test = []
commentlist_neu_test =[]
for comment in raw_data_list:
    commentid, videoid_raw, comment_text = comment
    
    if commentid in predictiondict:
        
        label,videoid,comment_output = predictiondict[commentid]
        
        if videoid == videoid_raw:
            
            raw_comment = comment_text
            comment = raw_comment.lower().strip()
            words = comment.split()
            
            if label == "positive":
                commentlist_pos_test.append(words)
                labellist_pos_test.append(label)
            elif label == "negative":
                commentlist_neg_test.append(words)
                labellist_neg_test.append(label)
            elif label == "neutral":
                commentlist_neu_test.append(words)
                labellist_neu_test.append(label)
        else:
            print(commentid, videoid_raw, comment_text )
    else:
        print(commentid)

In [16]:
print(len(commentlist_pos_test))
print(len(commentlist_neg_test))
print(len(commentlist_neu_test))

print(len(commentlist_pos_test)+len(commentlist_neg_test)+len(commentlist_neu_test))

263118
103297
1249227
1615642


In [17]:
lexiconfile = pd.read_csv('lexicons/SentiWordNet_3.0.0.txt', sep="\t")

In [18]:
lexiconfile.head(2)

,# POS,ID,PosScore,NegScore,SynsetTerms,Gloss
0,a,1740.0,0.125,0.00,able#1,(usually followed by `to') having the necessar...
1,a,2098.0,0.000,0.75,unable#1,(usually followed by `to') not having the nece...


In [19]:
pos = []
neg = []

for num, row in lexiconfile.iterrows():
    # add positives
    if row['PosScore'] > 0.6 and row['NegScore'] < 0.2:
        words = row['SynsetTerms'].split()
        for word in words: 
            word = word.rstrip('0123456789#')
            if '_' not in word:
                pos.append(word)
    # add negatives
    elif row['NegScore'] > 0.6 and row['PosScore'] < 0.2:
        words = row['SynsetTerms'].split()
        for word in words: 
            word = word.rstrip('0123456789#')
            if '_' not in word:
                neg.append(word)

print(len(pos), len(neg))

2310 4878


In [20]:
# Check if words occur in both lists, if they do check how many times
# if word occur more in one list remove in other, if equal remove from both lists

new_pos = []
new_neg = []

for word in pos:
    if word in neg:
        if pos.count(word) > neg.count(word):
            if word not in new_pos:
                new_pos.append(word)
        elif neg.count(word) > pos.count(word):
            if word not in new_neg:
                new_neg.append(word)
    else:
        if word not in new_pos:
            new_pos.append(word)

for word in neg:
    if word not in new_neg and word not in pos:
        new_neg.append(word)

In [21]:
print(len(new_pos),len(new_neg))

1962 4113


In [22]:
# Add _pos to positive words

new_pos_comments = []
changed_words_pos = []

for line in commentlist_pos_test:
    new_line = []
    for word in line:
        if word in new_pos:
            
            new_word = word +"_pos"
            
#             count_changes_pos[word] += 1

            new_line.append(new_word)
            changed_words_pos.append(word)
            
        else:
            new_line.append(word)
    new_pos_comments.append(new_line)

print(new_pos_comments[:100])
print(len(changed_words_pos))

[['i', 'have', 'a', 'huion', 'and', 'it', 'is', 'massive!!!!!!', 'i', "don't", 'know_pos', 'whether', "it's", 'just_pos', 'me', 'cos', "i'm", 'a', 'midget', 'and', 'like_pos', '11', 'soooooo', 'lol'], ['what', 'was', 'the', 'one', 'you', 'use', 'normally', 'called?', "i'd", 'love_pos', 'a', 'cheapish', 'tablet', 'for', 'christmas'], ['i', 'love_pos', 'you,', 'you', 'are', 'literally', 'favorite', 'youtuber', 'right_pos', 'now', "everyone's", 'sooo', 'dramatic', '(except', 'joana', 'ceddia)', 'you', 'make_pos', 'my', 'day', '☺️'], ['i', 'love_pos', 'the', 'way', 'you', 'draw!', "i'm", 'getting', 'a', 'drawing', 'tablet', 'tomorrow', 'and', 'thinking', 'about', 'doing', 'animation.', 'any', 'tips?'], ['nice_pos', 'and', 'i', 'have', 'no', 'idea', 'how', 'to', 'do', 'digital', 'art', 'with', 'an', 'art', 'tablet', 'i', 'normally', 'do', 'digital', 'art', 'on', 'my', 'android', 'tablet', 'with', 'my', 'finger', 'and', 'i', 'need', 'to', 'get', 'a', 'stylist', 'for', 'it', 'so', 'yea', 'but

In [33]:
count_changes_pos = Counter(changed_words_pos)

In [34]:
# Add _neg to positive words

new_neg_comments = []
changed_words_neg = []
count_changes_neg = 0

for line in commentlist_neg_test:
    new_line = []
    for word in line:
        if word in new_neg:
            new_word = word +"_neg"
            count_changes_neg += 1
            new_line.append(new_word)
            changed_words_neg.append(word)
        else:
            new_line.append(word)
    
    new_neg_comments.append(new_line)

print(new_neg_comments[:10])
print(len(changed_words_neg))

[['this', 'piece', 'resonates', 'with', 'me', 'too.', "it's", 'very', 'simple', 'melodically,', 'harmonically,', 'and', 'formally', 'which', 'is', 'why', 'i', 'like', 'it.', 'it', 'starts', 'out_neg', 'with', 'a', 'descending', 'dominant', 'to', 'tonic', 'relationship,', 'which', 'in', 'western', 'music', 'is', 'very', 'powerful.', "it's", 'the', 'basis', 'for', 'pretty', 'much', 'every', 'piece', 'of', 'music', 'you', 'hear.', 'after', 'that,', "it's", 'essentially', 'just', 'a', 'minor_neg', 'arpeggio', 'repeated', 'over', 'a', 'tonic', 'drone', 'with', 'an', 'occasional', 'second', 'scale', 'degree', 'that', 'sticks', 'out_neg', 'from', 'the', 'woodwinds.', 'formally', "it's", 'very', 'simple', 'too.', 'it', 'starts', 'off_neg', 'calm,', 'becomes', 'a', 'swirl', 'of', 'repeated', 'arpeggios', 'that', 'almost', 'sound', 'out_neg', 'of', 'sync,', 'then', 'returns', 'to', 'a', 'state', 'of', 'calmness.', 'i', 'almost', 'wish', 'this', 'piece', 'were', 'longer,', 'but', 'i', 'think', 'm

In [430]:
count_changes_neg = Counter(changed_words_neg)

In [431]:
# Remove double words
words_neg = list(OrderedDict.fromkeys(changed_words_neg))
words_pos = list(OrderedDict.fromkeys(changed_words_pos))

In [432]:
print(len(words_pos))

940


In [433]:
# pickle.dump(words_neg, open("changed_words_neg_ALL.p", "wb" ))
# pickle.dump(words_pos, open("changed_words_pos_ALL.p", "wb" ))

In [434]:
new_data = new_neg_comments + new_pos_comments + commentlist_neu_test

In [435]:
# Count amount of times each changed word occurs in other context for weighting the embeddings

changed_word_neg_other_context = [] 
changed_word_pos_other_context = [] 

for line in new_data:
    for word in line:
        if word in words_neg:
            changed_word_neg_other_context.append(word)
        elif word in words_pos:
            changed_word_pos_other_context.append(word)

In [436]:
count_changes_neg_other = Counter(changed_word_neg_other_context)
count_changes_pos_other = Counter(changed_word_pos_other_context)

In [437]:
def getWeightdict(count_changes_pos,count_changes_pos_other):
    pos_word_weights = {}
    for word in count_changes_pos:
        pos_count = count_changes_pos[word]
        other_count = count_changes_pos_other[word]
        total_count_pos = pos_count + other_count
        weight_pos = pos_count / total_count_pos
        weight_other_pos = other_count / total_count_pos
        pos_word_weights[word] = [weight_pos,weight_other_pos]
    return pos_word_weights 

In [438]:
pos_word_weights = getWeightdict(count_changes_pos,count_changes_pos_other)

In [439]:
neg_word_weights = getWeightdict(count_changes_neg,count_changes_neg_other)

In [443]:
# Train both skip models

# model_SKIP = Word2Vec(new_data, size=300, sg=1)
model_SKIP_n10 = Word2Vec(new_data, size=300, sg=1, negative=10)

In [451]:
model_SKIP_n10.most_similar("great_pos")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('fantastic', 0.7516839504241943),
 ('awesome_pos', 0.7325202822685242),
 ('great', 0.7270063161849976),
 ('good_pos', 0.7152231931686401),
 ('terrific', 0.7093876600265503),
 ('excellent_pos', 0.6888229846954346),
 ('amazing_pos', 0.6559268236160278),
 ('splendid_pos', 0.643805205821991),
 ('excelent', 0.6415426135063171),
 ('wonderful_pos', 0.6415295600891113)]

In [447]:
model_SKIP_n10.wv.vectors.shape[0]

115221

In [448]:
model_SKIP_n10.save("SKIP_negative10_EN_Change_inputSentiment_no_delete_test.model")

In [459]:
# Delete other context words

model_dict_delete = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0


for word in model_SKIP.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos":
        amount_posneg +=1
        non_word = word[:-4]
        if non_word not in model_dict_delete:
            model_dict_delete[non_word] = model_SKIP_n10.wv[word]
        else:
            print(word)
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_delete:
                model_dict_delete[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1

print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)

115221

1942
1406
111873

115221

0
0


In [460]:
print(len(model_dict_delete))

113279


In [457]:
print(model_dict_delete["great"])

[-2.59416968e-01  3.22017878e-01  1.24966644e-01 -7.26944255e-03
  1.18611142e-01  2.84944832e-01 -3.74072611e-01  1.82498217e-01
 -1.11310765e-01 -2.04666793e-01  5.33664133e-03 -4.21804249e-01
 -8.36584121e-02 -1.55637532e-01  7.96585456e-02 -4.77901800e-03
 -8.92233253e-02 -1.60887465e-01  1.51048526e-01  3.95493299e-01
 -2.19169870e-01  1.03375562e-01 -1.09550962e-02 -1.94786295e-01
  4.10436839e-02  2.37075165e-02  4.59755920e-02 -2.44538523e-02
  2.75693871e-02  2.10444123e-01 -2.60958206e-02  2.06231236e-01
  2.06388071e-01  7.50712231e-02 -9.67616886e-02 -1.23695638e-02
  3.12847614e-01 -1.77890837e-01 -5.69908693e-02  1.46759897e-01
  1.72496021e-01 -3.67852822e-02 -1.40052829e-02  7.99474120e-02
  1.43799737e-01  6.53669704e-04  1.33343309e-01 -4.57338154e-01
  1.50974497e-01 -1.29882336e-01 -2.46428549e-02  1.79461658e-01
  1.72829419e-01 -8.70544761e-02 -4.05319408e-02  2.69188076e-01
  6.90854117e-02 -1.07866108e-01  8.72672200e-02  4.59691882e-02
 -1.35516748e-01 -2.90446

In [461]:
pickle.dump(model_dict_delete, open("SKIP_EN_Change_inputSentiment_deletedoubles.p", "wb" ))

In [465]:
# Average the embeddings

model_dict_average = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0


for word in model_SKIP.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos":
        amount_posneg +=1
        non_word = word[:-4]
        
        vectorslist = []
        posneg_vector = model_SKIP_n10.wv[word]
        vectorslist.append(posneg_vector)
        
        try:
            other_vector = model_SKIP_n10.wv[non_word]
            vectorslist.append(other_vector)
            
            vectorarray = np.array(vectorslist)
            averagevector = np.average(vectorarray, axis=0)
            
            model_dict_average[non_word] = model_SKIP_n10.wv[averagevector]
            
        except:
            # use posneg word
            model_dict_average[non_word] = model_SKIP_n10.wv[word]
    
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_average:
                model_dict_average[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1
        
        
            
        

print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)

115221

1942
1406
111873

115221

0
0


In [466]:
pickle.dump(model_dict_average, open("SKIP_EN_Change_inputSentiment_average.p", "wb" ))

In [469]:
# Weight and sum embeddings

model_dict_weightsum = {}

count_word_del = 0
count_all = 0
count_else = 0
amount_posneg = 0
indict = 0

for word in model_SKIP.wv.vocab:
    count_all +=1
    if word[-4:] == "_neg" or word[-4:] == "_pos": 
        amount_posneg +=1
        non_word = word[:-4]
        
        if word[-4:] == "_neg":
            sentiment_weight, other_weight = neg_word_weights[non_word]
        else:
            sentiment_weight, other_weight = pos_word_weights[non_word]
        
        vectorslist = []
        posneg_vector = model_SKIP_n10.wv[word]
        vectorslist.append(posneg_vector*sentiment_weight)
        
        try:
            other_vector = model_SKIP_n10.wv[non_word]
            vectorslist.append(other_vector*other_weight)
            
            vectorarray = np.array(vectorslist)
            weight_sum_vector = np.sum(vectorarray, axis=0)
            
            model_dict_weightsum[non_word] = model_SKIP_n10.wv[weight_sum_vector]
        except:
            # use posneg word
            model_dict_weightsum[non_word] = model_SKIP_n10.wv[word]
    
    else:
        if word not in words_neg and word not in words_pos:
            count_else +=1
            if word not in model_dict_weightsum:
                model_dict_weightsum[word] = model_SKIP_n10.wv[word]
            else:
                print(word)
                indict  +=1
        else:
            count_word_del +=1
        
        
            
print(count_all)
print("")   
print(count_word_del)
print(amount_posneg)
print(count_else)
print("")
print(count_word_del + amount_posneg + count_else)

115221

1942
1406
111873

115221

0
0


In [470]:
pickle.dump(model_dict_weightsum, open("SKIP_EN_Change_inputSentiment_weightsum.p", "wb" ))